In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
allrois_inibi = pd.read_csv("data/inifak xp2 com exclude outside.csv")

In [3]:
allrois_inibi.head()

,label,Area,id_gh2ax,id_ku80,id_fak,id_dapi,tto
0,DOX12-DMSO12-gH2AX-647-ku80-546-fak-488-dapi-4...,2.599,67022.453,30757.229,20350.634,86727.865,doxdmso
1,DOX12-DMSO12-gH2AX-647-ku80-546-fak-488-dapi-4...,2.433,49921.257,33581.841,26658.237,160632.634,doxdmso
2,DOX12-DMSO12-gH2AX-647-ku80-546-fak-488-dapi-4...,2.883,60990.843,32400.324,22427.352,64870.387,doxdmso
3,DOX12-DMSO12-gH2AX-647-ku80-546-fak-488-dapi-4...,2.339,33093.914,26838.394,19945.921,62112.065,doxdmso
4,DOX12-DMSO12-gH2AX-647-ku80-546-fak-488-dapi-4...,4.171,72552.883,55653.885,30931.689,129398.437,doxdmso


In [4]:
pd.options.display.max_colwidth = 200
allrois_inibi['label']

0       DOX12-DMSO12-gH2AX-647-ku80-546-fak-488-dapi-405-01_Out_Channel Alignment.czi_nucleo_0(6).tifnucleo_0-roi_0.tiff
1       DOX12-DMSO12-gH2AX-647-ku80-546-fak-488-dapi-405-01_Out_Channel Alignment.czi_nucleo_0(6).tifnucleo_0-roi_1.tiff
2       DOX12-DMSO12-gH2AX-647-ku80-546-fak-488-dapi-405-01_Out_Channel Alignment.czi_nucleo_1(6).tifnucleo_0-roi_0.tiff
3       DOX12-DMSO12-gH2AX-647-ku80-546-fak-488-dapi-405-01_Out_Channel Alignment.czi_nucleo_1(6).tifnucleo_0-roi_1.tiff
4       DOX12-DMSO12-gH2AX-647-ku80-546-fak-488-dapi-405-01_Out_Channel Alignment.czi_nucleo_1(6).tifnucleo_0-roi_2.tiff
                                                              ...                                                       
1630      DOX12-PF12-gH2AX-647-ku80-546-fak-488-dapi-405-21_Out_Channel Alignment.czi_nucleo_2(9).tifnucleo_0-roi_5.tiff
1631      DOX12-PF12-gH2AX-647-ku80-546-fak-488-dapi-405-21_Out_Channel Alignment.czi_nucleo_2(9).tifnucleo_0-roi_6.tiff
1632      DOX12-PF12-gH2AX-647-k

In [5]:
# Extract the number before "_Out_Channel"
allrois_inibi['image'] = allrois_inibi['label'].str.extract(r'(\d+)_Out_Channel')

# Extract the number after "czi_nucleo_"
allrois_inibi['nucleo'] = allrois_inibi['label'].str.extract(r'czi_nucleo_(\d+)')

# Extract the number after "roi_"
allrois_inibi['roi'] = allrois_inibi['label'].str.extract(r'roi_(\d+)')

# Remove the string part ".tifnucleo_0"
allrois_inibi['label'] = allrois_inibi['label'].str.replace('.tifnucleo_0', '')

C:\Users\the_s\AppData\Local\Temp\ipykernel_8216\1624936641.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  allrois_inibi['label'] = allrois_inibi['label'].str.replace('.tifnucleo_0', '')


In [6]:
allrois_inibi.to_csv("data/inifak_exp2_clean.csv")

In [7]:
df  = pd.read_csv("data/inifak_exp2_clean.csv")

In [8]:
tto_list = ['doxdmso', 'doxpf']

pc_df = pd.DataFrame()

for tto in tto_list:
    image_list = df[(df['tto'] == tto)]["image"].unique()
    for image in image_list:
        nucleos_list = df[(df['tto'] == tto) & (df['image'] == image)]['nucleo'].unique()
        for nucleo in nucleos_list:
            id_ku80 = df[(df['tto'] == tto) & (df['image'] == image) & (df['nucleo'] == nucleo)]['id_ku80']
            id_gh2ax = df[(df['tto'] == tto) & (df['image'] == image) & (df['nucleo'] == nucleo)]['id_gh2ax']
            id_fak = df[(df['tto'] == tto) & (df['image'] == image) & (df['nucleo'] == nucleo)]['id_fak']
            total_rois = df[(df['tto'] == tto) & (df['image'] == image) & (df['nucleo'] == nucleo)].shape[0]
        
            pearson_id_gh2ax_id_ku80 = np.corrcoef(id_gh2ax, id_ku80)[0][1]
            pearson_id_fak_id_ku80 = np.corrcoef(id_fak, id_ku80)[0][1]
            pearson_id_fak_id_gh2ax = np.corrcoef(id_fak, id_gh2ax)[0][1]
            
            df_rcorr = pd.DataFrame(np.transpose([[image],[tto], [nucleo], [total_rois], [pearson_id_gh2ax_id_ku80], [pearson_id_fak_id_ku80], [pearson_id_fak_id_gh2ax]]), 
                                    columns=['image','tto', 'nucleo', 'nrois', 'pc_gh2ax_ku80', 'pc_fak_ku80', 'pc_gh2ax_fak'])
            pc_df = pd.concat([pc_df, df_rcorr], ignore_index=True)

C:\Users\the_s\anaconda3\lib\site-packages\numpy\lib\function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\the_s\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\Users\the_s\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
C:\Users\the_s\anaconda3\lib\site-packages\numpy\lib\function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\the_s\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\Users\the_s\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
C:\Users\the_s\anacond

In [9]:
pc_df

,image,tto,nucleo,nrois,pc_gh2ax_ku80,pc_fak_ku80,pc_gh2ax_fak
0,1,doxdmso,0,2,-1.0,0.9999999999999998,-1.0
1,1,doxdmso,1,8,0.8903683139442075,0.9595189998252693,0.957507029623281
2,2,doxdmso,0,88,0.910313023042677,0.9573785628015523,0.9600562014998832
3,2,doxdmso,1,4,-0.6116496462742367,-0.48642305194889585,0.20193731301458928
4,3,doxdmso,0,11,0.9092440135149636,0.9387036788382782,0.964816745654254
...,...,...,...,...,...,...,...
84,20,doxpf,2,10,0.565292311394325,0.9662718677041789,0.7266706637902351
85,20,doxpf,3,22,-0.17246412100838127,0.8516752699202128,0.30756158170293896
86,21,doxpf,0,4,0.6570729954810626,0.9942169862121317,0.6503130433589484
87,21,doxpf,1,20,0.16236989162047014,0.7011461850353156,0.6040033315054243


In [10]:
pc_df['image'] = pc_df['image'].astype(int)
pc_df = pc_df.sort_values(['tto','image'])
pc_df.to_csv("data/inifak_exp2_cell_pc.csv")